# Chapter 5 - Ultimate † Machine

これまでの話：

- 3章4章では，単純な計算モデルの能力調べた (複雑な文字列認識・正規表現・パース)
- 有限オートマトンやプッシュダウン・オートマトンでは限界がある

この章の話：  
ハードコードされたタスクを実行するのではなく、プログラムを実行できる機械を設計したい！  
　→ 自動機械 (automatic machine) を設計する

## 5.1 決定性チューリングマシン

4章ではスタックを外部記憶として使ってたけど、スタックは格納した後のデータの利用方法が不便（First-In Last-Out）  
もっと柔軟なストレージに置き換えよう！

### 5.1.1 ストレージ

チューリングさんの案

- 機械に無限の長さの空白のテープを渡し，テープの任意の場所に文字を読み書きできるようにする
  - 無限の長さ？→ 必要に応じて両端を伸ばせる1次元配列
- テープはストレージと入力，両方の役割を果たす
  - 事前にそのテープを入力として扱う文字列で埋めておき，計算を実行しながらテープにある文字を読む
    - 必要に応じて上書きする

無限の長さのテープにアクセスする有限状態機械：  
$\underline{チューリングマシン {\rm (TM：Turing Machine)}}$ ( a.k.a $\underline{決定性チューリングマシン {\rm (Deterministic Turing Machine)}}$ )

↓ どうやって設計するの？

- 選択肢1つ目：各マス目にユニークな文字からなるアドレスをつけ、テープをランダムアクセス可能にする
  - 無限の長さのテープのどうやってアドレスを割り当てるのか
  - アクセスしたいマス目のアドレスをどうやって指定するのか
- 選択肢2つ目：テープ上の特定の位置 ($\underline{テープヘッド {\rm (tape head)}}$) にある文字だけを読み書きすることができることにする
  - 計算を1ステップ実行するたびに，テープヘッドを1マスずつ左右どちらかに動かせる
    - 離れた場所だと時間がかかるが，確実にデータにアクセスできる

選択肢2を採用！

#### 2進数をインクリメントするDTMを設計

状態が3つ (1/2/3) あって、状態3が受理状態  
テープヘッドが最も右にある数を指している状態がスタート状態 (状態1)  
最終状態ではテープヘッドをスタート状態にする

- 状態：1
  - 数が0であれば1で置き換えて、テープヘッドを右に動かし、状態2に進む
  - 数が1であれば0で置き換えて、その左にある数をインクリメントする
- 状態：2
  - 0か1を読むと、テープヘッドを右に動かす
  - 空白を読むと、テープヘッドを左に動かし、状態3に進む
- 状態：3
  - 受理

※表5 - 1参照した方が早い  
※テープヘッドを最初の位置に戻すのは、機械を再実行可能にするため

### 5.1.2 規則

上記であげた規則の形式は、以下の5つのパーツから構成される

- 機械の現在の状態
- テープヘッドの現在の位置にあるはずの文字
- 機械の次の状態
- テープヘッドの現在の位置に書くべき文字
- テープに書いたあと、テープヘッドを動かす方向（左右）

矢印にラベルがついてるだけで，DFAとほとんど同じ感じになる

#### 文字列認識

1個以上の文字 $a$ に、$b$ と $c$ が同じ数だけ続くような入力 ( \`$aaabbbccc$ \`) を認識する問題を考える  
以下を繰り返すことになる

- 状態：1
  - $a$ が見つかるまで、テープヘッドを右に動かして入力文字列をスキャン
  - $a$ を見つけたらそれを $X$ で置き換え，状態2に進む
  - $a$ を見つけられないまま末尾にまで達したら状態6に進む
- 状態：2
  - $b$ が見つかるまで、テープヘッドを右に動かして入力文字列をスキャン
  - $b$ を見つけたらそれを $X$ で置き換え，状態3に進む
- 状態：3
  - $c$ が見つかるまで、テープヘッドを右に動かして入力文字列をスキャン
  - $b$ を見つけたらそれを $X$ で置き換え、状態4に進む
- 状態：4
  - 入力文字の末尾を探して右にスキャンする
  - 入力の末尾に達したら状態5に進む
- 状態：5
  - 先頭を探して左にスキャンする
  - 先頭に達したら状態1に進む
- 状態：6
  - 受理
  
※表5 - 2参照した方が早い  
※別の文字が含まれることを想定していないので注意が必要

### 5.1.3 決定性

- 自由移動がない
- 状態と文字の組合せに対して矛盾がない規則を1つだけ持てる
  - 適用できる規則がないときいは暗黙の行き詰まりに陥る

### 5.1.4 シミュレーション

In [1]:
# チューリングマシンのテープの実装
#    - テープに書かれている文字の格納 (left - middle - right)
#    - テープヘッドの現在の位置 (middle)
#    - 無限に長くて空白のマス目で埋められていることを表現 (blank)

class Tape < Struct.new(:left, :middle, :right, :blank)
  def inspect
    "#<Tape #{left.join} (#{middle}) #{right.join}>"
  end
end

:inspect

In [2]:
# 新しいテープを作成
tape = Tape.new(['1', '0', '1'], '1', [], '_')

#<Tape 101 (1) >

In [3]:
# テープヘッドの下にある文字を読む
tape.middle

"1"

In [4]:
class Tape
  #テープヘッドの位置に文字を書く操作
  def write(character)
    Tape.new(left, character, right, blank)
  end
  
  # テープヘッドを左に動かす操作
  def move_head_left
    Tape.new(left[0..-2], left.last || blank, [middle] + right, blank)
  end
  
  # テープヘッドを右に動かす操作
  def move_head_right
    Tape.new(left + [middle], right.first || blank, right.drop(1), blank)
  end
end

:move_head_right

In [5]:
tape

#<Tape 101 (1) >

In [6]:
# テープヘッドを左に動かす
tape.move_head_left

#<Tape 10 (1) 1>

In [7]:
# テープヘッドの位置に `0` と書き込む
tape.write('0')

#<Tape 101 (0) >

In [8]:
# テープヘッドを右に動かす
tape.move_head_right

#<Tape 1011 (_) >

In [9]:
# テープヘッドの位置に `0` と書き込む
tape.move_head_right.write('0')

#<Tape 1011 (0) >

チューリングマシンの構成は状態とテープの組み合わせになる  
チューリングマシンの規則は，その構成を直接扱うようにして実装できる

In [10]:
# チューリングマシンの構成

class TMConfiguration < Struct.new(:state, :tape)
end

class TMRule < Struct.new(:state, :character, :next_state, :write_character, :direction)
  def applies_to?(configuration)
    state == configuration.state && character == configuration.tape.middle
  end
end

:applies_to?

In [11]:
rule = TMRule.new(1, '0', 2, '1', :right)

#<struct TMRule state=1, character="0", next_state=2, write_character="1", direction=:right>

In [12]:
# それぞれの規則は、現在の状態と現在テープヘッドの下にある文字が期待しているものと一致したときにのみ適用される

puts(rule.applies_to?(TMConfiguration.new(1, Tape.new([], '0', [], '_'))))
puts(rule.applies_to?(TMConfiguration.new(1, Tape.new([], '1', [], '_'))))
puts(rule.applies_to?(TMConfiguration.new(2, Tape.new([], '0', [], '_'))))

true
false
false


 規則にしたがって，新しい文字を書き，テープヘッドを動かし，機械の状態を変更する  
　　--> 機械の構成を更新

In [13]:
class TMRule
  def follow(configuration)
    TMConfiguration.new(next_state, next_tape(configuration))
  end
  
  def next_tape(configuration)
    written_tape = configuration.tape.write(write_character)
    
    case direction
    when :left
      written_tape.move_head_left
    when :right
      written_tape.move_head_right
    end
  end
end

:next_tape

In [14]:
rule.follow(TMConfiguration.new(1, Tape.new([], '0', [], '_')))

#<struct TMConfiguration state=2, tape=#<Tape 1 (_) >>

DTMRulebookの実装は，DFARulebookやDPDARulebookとほとんど同じ

In [15]:
class DTMRulebook < Struct.new(:rules)
  def next_configuration(configuration)
    rule_for(configuration).follow(configuration)
  end
  def rule_for(configuration)
    rules.detect { |rule| rule.applies_to?(configuration) }
  end
end

:rule_for

In [16]:
rulebook = DTMRulebook.new([
  TMRule.new(1, '0', 2, '1', :right),
  TMRule.new(1, '1', 1, '0', :left),
  TMRule.new(1, '_', 2, '1', :right),
  TMRule.new(2, '0', 2, '0', :right),
  TMRule.new(2, '1', 2, '1', :right),
  TMRule.new(2, '_', 3, '_', :left)
  ])

#<struct DTMRulebook rules=[#<struct TMRule state=1, character="0", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=1, character="1", next_state=1, write_character="0", direction=:left>, #<struct TMRule state=1, character="_", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="0", next_state=2, write_character="0", direction=:right>, #<struct TMRule state=2, character="1", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="_", next_state=3, write_character="_", direction=:left>]>

In [17]:
configuration = TMConfiguration.new(1, tape)

#<struct TMConfiguration state=1, tape=#<Tape 101 (1) >>

In [18]:
configuration = rulebook.next_configuration(configuration)

#<struct TMConfiguration state=1, tape=#<Tape 10 (1) 0>>

In [19]:
configuration = rulebook.next_configuration(configuration)

#<struct TMConfiguration state=1, tape=#<Tape 1 (0) 00>>

In [20]:
configuration = rulebook.next_configuration(configuration)

#<struct TMConfiguration state=2, tape=#<Tape 11 (0) 0>>

うまく実装出来ているっぽい！

DTMクラスにまとめて，#stepメソッドと#runメソッドを用意する

In [21]:
class DTM < Struct.new(:current_configuration, :accept_status, :rulebook)
  def accepting?
    accept_status.include?(current_configuration.state)
  end
  
  def step
    self.current_configuration = rulebook.next_configuration(current_configuration)
  end
  
  def run
    step until accepting?
  end
end

:run

In [22]:
dtm = DTM.new(TMConfiguration.new(1, tape), [3], rulebook)

#<struct DTM current_configuration=#<struct TMConfiguration state=1, tape=#<Tape 101 (1) >>, accept_status=[3], rulebook=#<struct DTMRulebook rules=[#<struct TMRule state=1, character="0", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=1, character="1", next_state=1, write_character="0", direction=:left>, #<struct TMRule state=1, character="_", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="0", next_state=2, write_character="0", direction=:right>, #<struct TMRule state=2, character="1", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="_", next_state=3, write_character="_", direction=:left>]>>

In [23]:
dtm.current_configuration

#<struct TMConfiguration state=1, tape=#<Tape 101 (1) >>

In [25]:
dtm.accepting?

false

In [26]:
dtm.step; dtm.current_configuration

#<struct TMConfiguration state=1, tape=#<Tape 10 (1) 0>>

In [27]:
dtm.accepting?

false

In [28]:
dtm.run

In [29]:
dtm.current_configuration

#<struct TMConfiguration state=3, tape=#<Tape 110 (0) _>>

In [30]:
dtm.accepting?

true

チューリングマシンには外部入力がないため，規則集と現在の構成を見るだけで，行き詰まり状態にあるかどうか判断できる

In [31]:
class DTMRulebook
  def applies_to?(configuration)
    !rule_for(configuration).nil?
  end
end

class DTM
  def stuck?
    !accepting? && !rulebook.applies_to?(current_configuration)
  end
  
  def run
    step until accepting? || stuck?
  end
end

:run

In [32]:
dtm = DTM.new(TMConfiguration.new(1, tape), [3], rulebook)

#<struct DTM current_configuration=#<struct TMConfiguration state=1, tape=#<Tape 101 (1) >>, accept_status=[3], rulebook=#<struct DTMRulebook rules=[#<struct TMRule state=1, character="0", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=1, character="1", next_state=1, write_character="0", direction=:left>, #<struct TMRule state=1, character="_", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="0", next_state=2, write_character="0", direction=:right>, #<struct TMRule state=2, character="1", next_state=2, write_character="1", direction=:right>, #<struct TMRule state=2, character="_", next_state=3, write_character="_", direction=:left>]>>

In [33]:
dtm.run

In [34]:
dtm.current_configuration

#<struct TMConfiguration state=3, tape=#<Tape 110 (0) _>>

In [35]:
dtm.accepting?

true

In [36]:
dtm.stuck?

false

stuck と accepting の出力が逆。。。

\`$aaabbbccc$\` のような文字列を認識するためのチューリングマシンを構築

In [37]:
rulebook = DTMRulebook.new([
  # 状態1: a を探して右にスキャンする
  TMRule.new(1, 'X', 1, 'X', :right),  # Xをスキップ
  TMRule.new(1, 'a', 2, 'X', :right),  # a を消して，状態2に進む
  TMRule.new(1, '_', 6, '_', :left),    # 空白を見つけて，状態6 (受理状態) に進む
  
  # 状態2: b を探して右にスキャンする
  TMRule.new(2, 'a', 2, 'a', :right),  # a をスキップする
  TMRule.new(2, 'X', 2, 'X', :right),  # X をスキップする
  TMRule.new(2, 'b', 3, 'X', :right),  # b を消して，状態3に進む
  
  # 状態3: c を探して右にスキャンする
  TMRule.new(3, 'b', 3, 'b', :right),  # b をスキップする
  TMRule.new(3, 'X', 3, 'X', :right),  # X をスキップする
  TMRule.new(3, 'c', 4, 'X', :right),   # c を消して，状態4に進む
  
  # 状態4: 文字列の末尾を探して右にスキャン
  TMRule.new(4, 'c', 4, 'c', :right),  # c をスキップする
  TMRule.new(4, '_', 5, '_', :left),   # 空白を見つけて，状態5に進む
  
  # 状態5: 文字列の先頭を探して左にスキャンする
  TMRule.new(5, 'a', 5, 'a', :left),   # a をスキップする
  TMRule.new(5, 'b', 5, 'b', :left),   # b をスキップする
  TMRule.new(5, 'c', 5, 'c', :left),   # c をスキップする
  TMRule.new(5, 'X', 5, 'X', :left),   # X をスキップする
  TMRule.new(5, '_', 1, '_', :right)  # 空白を見つけて，状態1に進む
  ])

#<struct DTMRulebook rules=[#<struct TMRule state=1, character="X", next_state=1, write_character="X", direction=:right>, #<struct TMRule state=1, character="a", next_state=2, write_character="X", direction=:right>, #<struct TMRule state=1, character="_", next_state=6, write_character="_", direction=:left>, #<struct TMRule state=2, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=2, character="X", next_state=2, write_character="X", direction=:right>, #<struct TMRule state=2, character="b", next_state=3, write_character="X", direction=:right>, #<struct TMRule state=3, character="b", next_state=3, write_character="b", direction=:right>, #<struct TMRule state=3, character="X", next_state=3, write_character="X", direction=:right>, #<struct TMRule state=3, character="c", next_state=4, write_character="X", direction=:right>, #<struct TMRule state=4, character="c", next_state=4, write_character="c", direction=:right>, #<struct TMRule state=4, characte

In [38]:
tape = Tape.new([], 'a', ['a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'], '_')

#<Tape  (a) aabbbccc>

In [39]:
dtm = DTM.new(TMConfiguration.new(1, tape), [6], rulebook)

#<struct DTM current_configuration=#<struct TMConfiguration state=1, tape=#<Tape  (a) aabbbccc>>, accept_status=[6], rulebook=#<struct DTMRulebook rules=[#<struct TMRule state=1, character="X", next_state=1, write_character="X", direction=:right>, #<struct TMRule state=1, character="a", next_state=2, write_character="X", direction=:right>, #<struct TMRule state=1, character="_", next_state=6, write_character="_", direction=:left>, #<struct TMRule state=2, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=2, character="X", next_state=2, write_character="X", direction=:right>, #<struct TMRule state=2, character="b", next_state=3, write_character="X", direction=:right>, #<struct TMRule state=3, character="b", next_state=3, write_character="b", direction=:right>, #<struct TMRule state=3, character="X", next_state=3, write_character="X", direction=:right>, #<struct TMRule state=3, character="c", next_state=4, write_character="X", direction=:right>, #

In [40]:
10.times { dtm.step }; dtm.current_configuration

#<struct TMConfiguration state=5, tape=#<Tape XaaXbbXc (c) _>>

In [41]:
25.times { dtm.step }; dtm.current_configuration

#<struct TMConfiguration state=5, tape=#<Tape _XXa (X) XbXXc_>>

In [42]:
dtm.run; dtm.current_configuration

#<struct TMConfiguration state=6, tape=#<Tape _XXXXXXXX (X) _>>

## 5.2 非決定性チューリングマシン

Q. 非決定性はチューリングマシンの能力を高めるか？  
A. ノー (非決定性チューリングマシンは決定性チューリングマシンを超えることはできない)

- 決定性有限オートマトンと決定性チューリングマシンは非決定性をシミュレートできる
  - 有限オートマトン：1つの状態を使って複数の状態の組合せを表現できる
  - チューリングマシン：1本のテープを使って複数のテープの内容を格納できる
- プッシュダウン・オートマトンは例外
  - 1つのスタックを使って複数のスタックを同時に表現できない
  
決定性チューリングマシンはシミュレートする機械が取り得るすべての構成を，幅優先で調べることができる

## 5.3 最大の能力

チューリングマシンには，どんな拡張もシミュレートできる能力がある  
→ チューリングマシンを拡張してさらに能力を高めようとしても，必ず失敗する運命にある

チューリングマシンの拡張 (内部ストレージ，サブルーチン，複数のテープ，多次元のテープ) について，なぜそれが計算能力を高めることにならないのか見ていく

### 5.3.1 内部ストレージ

チューリングマシンには文字を「覚えておく」手段がないため，覚えていた文字に応じてアクションを実行するといった動作をとるのは不可能のように思える

→ チューリングマシンはすぐれた内部ストレージを持っている．それは「現在の状態」である

In [43]:
# 文字列の先頭にある文字を末尾にコピーする

rulebook = DTMRulebook.new([
  # 状態1: テープから最初の文字を読む
  TMRule.new(1, 'a', 2, 'a', :right),  # a を覚える
  TMRule.new(1, 'b', 3, 'b', :right),  # b を覚える
  TMRule.new(1, 'c', 4, 'c', :rihgt),  # c を覚える
  
  # 状態2: 文字列の末尾を探して右にスキャンする（a を覚えている）
  TMRule.new(2, 'a', 2, 'a', :right),  # a をスキップする
  TMRule.new(2, 'b', 2, 'b', :right),  # b をスキップする
  TMRule.new(2, 'c', 2, 'c', :right),  # c をスキップする
  TMRule.new(2, '_', 5, 'a', :right),  # 空白を見つけて，a を書く
  
  # 状態3: 文字列の末尾を探して右にスキャンする (b を覚えている)
  TMRule.new(3, 'a', 3, 'a', :right),  # a をスキップする
  TMRule.new(3, 'b', 3, 'b', :right),  # b をスキップする
  TMRule.new(3, 'c', 3, 'c', :right),  # c をスキップする
  TMRule.new(3, '_', 5, 'b', :right),  # 空白を見つけて，b を書く
  
  # 状態4: 文字列の末尾を探して右にスキャンする (c を覚えている)
  TMRule.new(4, 'a', 4, 'a', :right),  # a をスキップする
  TMRule.new(4, 'b', 4, 'b', :right),  # b をスキップする
  TMRule.new(4, 'c', 4, 'c', :right),  # c をスキップする
  TMRule.new(4, '_', 5, 'c', :right)   # 空白を見つけて，c を書く
  ])

#<struct DTMRulebook rules=[#<struct TMRule state=1, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=1, character="b", next_state=3, write_character="b", direction=:right>, #<struct TMRule state=1, character="c", next_state=4, write_character="c", direction=:rihgt>, #<struct TMRule state=2, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=2, character="b", next_state=2, write_character="b", direction=:right>, #<struct TMRule state=2, character="c", next_state=2, write_character="c", direction=:right>, #<struct TMRule state=2, character="_", next_state=5, write_character="a", direction=:right>, #<struct TMRule state=3, character="a", next_state=3, write_character="a", direction=:right>, #<struct TMRule state=3, character="b", next_state=3, write_character="b", direction=:right>, #<struct TMRule state=3, character="c", next_state=3, write_character="c", direction=:right>, #<struct TMRule state=3, charact

In [44]:
tape = Tape.new([], 'b', ['c', 'b', 'c', 'a'], '_')

#<Tape  (b) cbca>

In [45]:
dtm = DTM.new(TMConfiguration.new(1, tape), [5], rulebook)

#<struct DTM current_configuration=#<struct TMConfiguration state=1, tape=#<Tape  (b) cbca>>, accept_status=[5], rulebook=#<struct DTMRulebook rules=[#<struct TMRule state=1, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=1, character="b", next_state=3, write_character="b", direction=:right>, #<struct TMRule state=1, character="c", next_state=4, write_character="c", direction=:rihgt>, #<struct TMRule state=2, character="a", next_state=2, write_character="a", direction=:right>, #<struct TMRule state=2, character="b", next_state=2, write_character="b", direction=:right>, #<struct TMRule state=2, character="c", next_state=2, write_character="c", direction=:right>, #<struct TMRule state=2, character="_", next_state=5, write_character="a", direction=:right>, #<struct TMRule state=3, character="a", next_state=3, write_character="a", direction=:right>, #<struct TMRule state=3, character="b", next_state=3, write_character="b", direction=:right>, #<st

In [46]:
dtm.run; dtm.current_configuration.tape

#<Tape bcbcab (_) >

### 5.3.2 サブルーチン

機械が $\underline{サブルーチン {\rm (subroutine)}}$ を呼び出せると，規則集の設計は簡単になる  
→ 便利なだけで，機械の能力を高めるものではない

小さなチューリングマシンを複数つなげることで，より大きなチューリングマシンを構築できる  
→ サブルーチンを開始する状態と終了する状態それぞれに，コピーした小さな機械の開始状態と受理状態をマージする

In [47]:
# 「数をインクリメントする」機械を3つつなげて「数に3を足す」機械を構築する

def increment_rules(start_state, return_state)
  incrementing = start_state
  finishing = Object.new
  finished = return_state
  
  [
    TMRule.new(incrementing, '0', finishing, '1', :right),
    TMRule.new(incrementing, '1', incrementing, '0', :left),
    TMRule.new(incrementing, '_', finishing, '1', :right),
    TMRule.new(finishing, '0', finishing, '0', :right),
    TMRule.new(finishing, '1', finishing, '1', :right),
    TMRule.new(finishing, '_', finished, '_', :left)
    ]
end

:increment_rules

In [48]:
added_zero, added_one, added_two, added_three = 0, 1, 2, 3

[0, 1, 2, 3]

In [50]:
rulebook = DTMRulebook.new(
  increment_rules(added_zero, added_one) +
  increment_rules(added_one, added_two) +
    increment_rules(added_two, added_three)
)

#<struct DTMRulebook rules=[#<struct TMRule state=0, character="0", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=0, character="1", next_state=0, write_character="0", direction=:left>, #<struct TMRule state=0, character="_", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="0", next_state=#<Object:0x007fc8728db4d0>, write_character="0", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="1", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="_", next_state=1, write_character="_", direction=:left>, #<struct TMRule state=1, character="0", next_state=#<Object:0x007fc8728db1b0>, write_character="1", direction=:right>, #<struct TMRule state=1, character="1", next_state=1, write_character="0", direction=:left>, #<struct TMRule state=1

In [51]:
rulebook.rules.length

18

In [52]:
tape = Tape.new(['1', '0', '1'], '1', [], '_')

#<Tape 101 (1) >

In [54]:
dtm = DTM.new(TMConfiguration.new(added_zero, tape), [added_three], rulebook)

#<struct DTM current_configuration=#<struct TMConfiguration state=0, tape=#<Tape 101 (1) >>, accept_status=[3], rulebook=#<struct DTMRulebook rules=[#<struct TMRule state=0, character="0", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=0, character="1", next_state=0, write_character="0", direction=:left>, #<struct TMRule state=0, character="_", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="0", next_state=#<Object:0x007fc8728db4d0>, write_character="0", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="1", next_state=#<Object:0x007fc8728db4d0>, write_character="1", direction=:right>, #<struct TMRule state=#<Object:0x007fc8728db4d0>, character="_", next_state=1, write_character="_", direction=:left>, #<struct TMRule state=1, character="0", next_state=#<Object:0x007fc8728db1b0>, write_character="1", direction=:rig

In [55]:
dtm.run; dtm.current_configuration.tape

#<Tape 111 (0) _>

### 5.3.3 複数のテープ

独立したテープヘッドを備えたテープをもう1本追加することによって，チューリングマシンの能力は高まるか？  
→ 当てはまらない．テープには十分なスペースがあるため，複数のテープの内容を1本のテープに畳み込める

### 5.3.4 多次元のテープ

チューリングマシンに無限の大きさの2次元の格子を使って，テープヘッドを左右だけでなく上下に動かせるようにする  
→ 2次元の格子は，必ず1次元のテープでシミュレートできる (たとえば，1次元のテープを2本使う)  
→ 5.3.3